In [1]:
setwd('~/Documents/GitHub/dors/')

In [2]:
library(reticulate)
use_condaenv("dors")

In [3]:
library(devtools);

Loading required package: usethis



In [4]:
load_all();

Loading DORS


Attaching package: ‘testthat’


The following object is masked from ‘package:devtools’:

    test_file




In [5]:
db <- dbConnect(DORS::DORS(), CTX = 'distributed')

In [6]:
db$dors_client$dashboard_link

http://127.0.0.1:8787/status

In [7]:
dbWriteTable(db, "mtcars", mtcars)

None

In [8]:
db$tables

{'mtcars': <dask_sql.datacontainer.DataContainer>}

In [9]:
res <- dbGetQuery(db, "SELECT * FROM mtcars WHERE cyl > 4")

In [10]:
head(res)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AMC Javelin,15.2,8,304,150,3.15,3.435,17.30,0,0,3,2
Cadillac Fleetwood,10.4,8,472,205,2.93,5.250,17.98,0,0,3,4
Camaro Z28,13.3,8,350,245,3.73,3.840,15.41,0,0,3,4
Chrysler Imperial,14.7,8,440,230,3.23,5.345,17.42,0,0,3,4
Dodge Challenger,15.5,8,318,150,2.76,3.520,16.87,0,0,3,2
Duster 360,14.3,8,360,245,3.21,3.570,15.84,0,0,3,4


In [ ]:
f.groupby(df.passenger_count).trip_distance.mean().compute()"

In [11]:
dbWriteTable(db, "taxi", "./sample_taxi.parquet")

None

In [14]:
dbGetQuery(db, "select count(*) from taxi")

COUNT(*)
<dbl>
743660


In [15]:
res <- dbGetQuery(db, "select avg(trip_distance), passenger_count from taxi group by passenger_count")

In [16]:
res

"AVG(""taxi"".""trip_distance"")",passenger_count
<dbl>,<dbl>
1.845263,0
2.484745,1
2.344380,2
2.313976,3
2.346265,4
2.325405,5
2.273989,6
0.000000,9
